In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
g = open('goodreads_preprocessed','rb')
goodreads = pickle.load(g)
g.close()

In [ ]:
i = open('imdb_preprocessed','rb')
imdb = pickle.load(i)
i.close()

# Descriptions

## Goodreads

In [ ]:
goodreads.head()

In [ ]:
sum_lis = []
for i in range(len(goodreads)):
    try:
        sum_lis.append(len(goodreads['Description'][i]))
    except:
        pass
print('Average length of description: ',sum(sum_lis)/len(sum_lis))

In [ ]:
#sample_desc_g = "Alternate Cover Edition ISBN: 0743273567 (ISBN13: 9780743273565)The Great Gatsby, F. Scott Fitzgerald's third book, stands as the supreme achievement of his career. This exemplary novel of the Jazz Age has been acclaimed by generations of readers. The story of the fabulously wealthy Jay Gatsby and his love for the beautiful Daisy Buchanan, of lavish parties on Long Island at a time when The New York Times noted ""gin was the national drink and sex the national obsession,"" it is an exquisitely crafted tale of America in the 1920s.The Great Gatsby is one of the great classics of twentieth-century literature.(from the back cover)"
sample_desc_g = "Harry Potter thinks he is an ordinary boy - until he is rescued by an owl, taken to Hogwarts School of Witchcraft and Wizardry, learns to play Quidditch and does battle in a deadly duel. The Reason ... HARRY POTTER IS A WIZARD!"


## IMDb

In [ ]:
imdb.head(3)

In [ ]:
sum_lis = []
for i in range(len(imdb)):
    try:
        sum_lis.append(len(imdb['Description'][i]))
    except:
        pass
print('Average length of description: ',sum(sum_lis)/len(sum_lis))

In [ ]:
#sample_desc_i = 'Although she is married to another man, Gatsby, a millionaire, woos his dream girl.'
sample_desc_i ="Adaptation of the first of J.K. Rowling's popular children's novels about Harry Potter, a boy who learns on his eleventh birthday that he is the orphaned son of two powerful wizards and possesses unique magical powers of his own. He is summoned from his life as an unwanted child to become a student at Hogwarts, an English boarding school for wizards. There, he meets several friends who become his closest allies and help him discover the truth about his parents' mysterious deaths."


# Sentence Similarity methods

In [ ]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
Input_Movie = 'The Great Gatsby'

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
imdb['stem_lem_Desc'] = imdb['Description'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w.lower()) for w in word_tokenize(x) if not w.lower() in stop_words]))

In [ ]:
imdb.head()

In [ ]:
sum_lis = []
for i in range(len(imdb)):
    try:
        sum_lis.append(len(imdb['stem_lem_Desc'][i]))
    except:
        pass
print('Average length of description: ',sum(sum_lis)/len(sum_lis))

In [ ]:
# Input_Movie = input('Enter: ').lower()
Input_Movie = 'the great gatsby'
Input_Description = imdb.loc[ imdb['primaryTitle'].apply(lambda x: x.lower()) == Input_Movie.lower() ,'stem_lem_Desc' ]
model = SentenceTransformer('all-mpnet-base-v2')


In [ ]:
sentences = list(imdb['stem_lem_Desc'].apply(lambda x: x.lower()))
sentences.insert(0,Input_Description)
sentence_embeddings = model.encode(sentences)
cos = cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:])
results = pd.DataFrame({'Test_Name': sentences[1:],'Similarity':cos.tolist()[0]})
results.sort_values('Similarity',ascending=False,inplace=True)
print(results.head())

In [ ]:
from gensim.models import KeyedVectors

# Load pre-trained Word2Vec model
w2v_model = KeyedVectors.load_word2vec_format('path_to_word2vec_model.bin', binary=True)
